In [1]:
import os
import s3fs
from aicsimageio.writers import OmeZarrWriter
from aicsimageio import AICSImage
from aicsimageio.dimensions import DimensionNames, DEFAULT_CHUNK_DIMS
import numpy

In [2]:

chunk_dims= [
    DimensionNames.SpatialY,
    DimensionNames.SpatialX,
    DimensionNames.Samples,
]
#filepath = "//allen/aics/assay-dev/MicroscopyData/Frick/2022/20220217/LLS/AD00000198/Lamin_multi-06-Deskew-28.czi"

#filepath = "//allen/programs/allencell/data/proj0/935/c72/962/3cc/7a4/37b/f4f/6d8/69c/91a/71/20200323_F01_001.czi"
#filename = "20200323_F01_001"

filepath = "E:\\data\\AICS-11_16415.ome.tif"
filename = "FOV_16415"

img = AICSImage(filepath, chunk_dims=chunk_dims)

02-Dec-22 09:25:22 - bfio.backends.PythonReader - WARNING  - read_metadata(): OME XML required reformatting.


In [3]:
from distributed import LocalCluster, Client
os.environ["AWS_PROFILE"] = "animatedcell"
os.environ["AWS_DEFAULT_REGION"] = "us-west-2"
#cluster = LocalCluster(processes=True)
cluster = LocalCluster(n_workers=4, processes=True, threads_per_worker=1)
# cluster = LocalCluster(memory_limit="7GB")  # threaded instead of multiprocess
# cluster = LocalCluster(n_workers=4, processes=True, threads_per_worker=1, memory_limit="12GB")
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 63.69 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:50877,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 63.69 GiB
Comm: tcp://127.0.0.1:50902,Total threads: 1
Dashboard: http://127.0.0.1:50907/status,Memory: 15.92 GiB
Nanny: tcp://127.0.0.1:50881,


In [4]:
# print some data about the image we loaded
scenes = img.scenes
print(scenes)
print(str(len(scenes)))

print(img.channel_names)
print(img.physical_pixel_sizes)



('Image:0',)
1
['CMDRP', 'EGFP', 'H3342', 'Bright_100X', 'SEG_STRUCT', 'SEG_Memb', 'SEG_DNA', 'CON_Memb', 'CON_DNA']
PhysicalPixelSizes(Z=0.29, Y=0.10833333333333332, X=0.10833333333333332)


In [6]:
# TODO: add generated thumbnail image with name "thumbnail" as 2d rgb data? or 3 channels?
# TODO compression?

# construct some per-channel lists to feed in to the writer.
# hardcoding to 9 for now
channel_colors = [0xff0000, 0x00ff00, 0x0000ff, 0xffff00, 0xff00ff, 0x00ffff, 0x880000, 0x008800, 0x000088]

s3 = s3fs.S3FileSystem(anon=False, config_kwargs={"connect_timeout": 60})

def write_scene(storeroot, scenename, sceneindex, img):
    print(scenename)
    print(storeroot)
    img.set_scene(sceneindex)
    pps = img.physical_pixel_sizes
    cn = img.channel_names

    data = img.get_image_dask_data("TCZYX")
    print(data.shape)
    writer = OmeZarrWriter(storeroot)

    writer.write_image(
        image_data=data,  # : types.ArrayLike,  # must be 5D TCZYX
        image_name=scenename,  #: str,
        physical_pixel_sizes=pps, # : Optional[types.PhysicalPixelSizes],
        channel_names=cn, # : Optional[List[str]],
        channel_colors=channel_colors, # : Optional[List[int]],
        scale_num_levels=4, # : int = 1,
        scale_factor=2.0, #  : float = 2.0,
    )    

# known scenes we want to write:
# babybear, goldilocks, mamabear, papabear
#scene_indices = [9-1,6-1,5-1,8-1]
# scene_indices = [9-1]
#scene_indices = [6-1,5-1,8-1]
scene_indices = range(len(img.scenes))

for i in scene_indices:
    scenename = img.scenes[i]
    scenename = scenename.replace(":","_")
    write_scene("s3://animatedcell-test-data/"+filename+"/"+scenename+".zarr/", scenename, i, img)


Image_0
s3://animatedcell-test-data/FOV_16415/Image_0.zarr/
(1, 9, 65, 624, 924)


In [16]:
#############################################
# READ BACK
#############################################
from ome_zarr.reader import Multiscales, Reader
from ome_zarr.io import parse_url

s3 = s3fs.S3FileSystem()
mypath = 's3://animatedcell-test-data/' + filename + "/" + scenes[scene_indices[0]] + ".zarr" 

reader = Reader(parse_url(mypath))
node = list(reader())[0]
# levels
print(len(node.data))
for i in range(len(node.data)):
    print(f"shape of level {i} : {node.data[i].shape}")
#for i in range(len(node.data)):
#    print("chunks ", node.data[i].chunks)


4
shape of level 0 : (570, 2, 42, 1248, 1824)
shape of level 1 : (570, 2, 42, 624, 912)
shape of level 2 : (570, 2, 42, 312, 456)
shape of level 3 : (570, 2, 42, 156, 228)


In [15]:
import nbvv
level = 1
levelxyscale = 2**(level+1)
t = 220
readdata = node.data[level][t][0:2].compute()
print(readdata.shape)
nbvv.volshow(readdata, spacing=(img.physical_pixel_sizes.X*levelxyscale,
        img.physical_pixel_sizes.Y*levelxyscale,
        img.physical_pixel_sizes.Z))

(2, 42, 624, 912)


VolumeWidget(dimensions={'tile_width': 341, 'tile_height': 256, 'rows': 8, 'cols': 6, 'atlas_width': 2046, 'at…